# 선 실행 코드

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 69.8 MB/s 
     |████████████████████████████████| 453 kB 63.7 MB/s 


In [ ]:
!pip install gdown

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from tqdm.notebook import tqdm

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

import os

import numpy as np

from datetime import datetime
import json
import re

from collections import Counter
import csv 
import time

# 크롤링

## 아스키코드 변환 함수

In [ ]:
# 한글 검색어(key)를 아스키코드로 변환하는 함수
def ascii(key):
  key_tag = []
  for i in tqdm(range(len(key))):                                       
    ts = key[i].split()
    if len(ts) == 1 :                                                   # 띄어쓰기가 없는 검색어(key)를 아키코드로 변환
      ts_par = parse.quote(ts[0])
      ascii_list.append(ts_par)
    elif len(ts) > 1:                                                   # 띄어쓰기가 된 검색어(key)를 아키코드로 변환 할때 아키코드+아키코드 로 만들기 위한 코드
      for j in ts:
        key_tag.append(parse.quote(j))
      if len(key_tag) == 2:                                                # 띄어쓰기가 1개일 때
        t_tag = key_tag[0] + '+' + key_tag[1]
        ascii_list.append(t_tag)
        key_tag.clear()
      elif len(key_tag) == 3:                                              # 띄어쓰기가 2개 일때
        t_tag = key_tag[0] + '+' + key_tag[1] + '+' + key_tag[2]                
        ascii_list.append(t_tag)
        key_tag.clear()
      elif len(key_tag) == 4:                                              # 띄어쓰기가 3개 일때
        t_tag = key_tag[0] + '+' + key_tag[1] + '+' + key_tag[2] + '+' + key_tag[3]   
        ascii_list.append(t_tag)
        key_tag.clear()
#=============================================================================

## 본문 크롤링 함수

테스트 진행시 전처리 부분으로 넘어가주세요

In [ ]:
# 언론사별 키워드로 본문등 크롤링
def news_trend(press):
# 한개의 언론사로 10개의 키워드를 차례대로 검색해서 나오는 제목 , 제목의 href, 날짜를 2022년으로 조건을 걸어서 저장
  for key in tqdm(press.keys()):   # 언론사 딕셔너리의 키값들
    temp_tags = []  # 중복 체크를 위한 임시 리스트
    
    for ascii_key in tqdm(ascii_list):     # 아스키코드로 변환된 키워드
      if press[key] == '디지털타임스':    # 특정 언론사에 대한 페이지 조건
        num = 0
      else:
        num = 1
      break_year = []
      for number in range(100):       # 페이지 스탑을 위한 포문
        url = press[key][0].format(num, ascii_key)          # 언론사 딕셔너리에 저장되어있는 0번째 url을 가지고 와서 num과 joy를 format함
        try:
          time.sleep(1)
          resp = requests.get(url, headers=headers)     # 저장된 url을 get으로 가져와서 resp에 저장
        except:
          time.sleep(3)
          resp = requests.get(url, headers=headers)     # 저장된 url을 get으로 가져와서 resp에 저장
        soup = BeautifulSoup(resp.content, 'lxml')    
        a_tit_tags = soup.select(press[key][1]) # 제목의 a태그
        a_day_tags = soup.select(press[key][2]) # 날짜 태그
        # 날짜에 따른 제목 a태그 가져오는 테스트

        for i in range(len(a_day_tags)):
          day = re.sub("[^0-9]", "", a_day_tags[i].text)

          if day[:4] == '2022' or day[:4] == '2021': # tag[i].text[15:19] 
            temp_tags.append(a_tit_tags[i]['href'])

          elif day[:4] == '2020':
            break_year.append(day[:4])
            n = 1
            break

        if '2020' in break_year:
          break
        else:
          if press[key] == '디지털타임스':
            num += 20
          else :
            num += 1

      # 2. 가져온 href가 중복인지 거르고 
    overlap = list(set(temp_tags))
    

    for l in tqdm(range(len(overlap))):
      
      try:
        time.sleep(1)
        if overlap[l][0] == 'h':
          href_url = overlap[l]
        else:
          href_url = press[key][3]+overlap[l]
      except:
        time.sleep(3)
        if overlap[l][0] == 'h':
          href_url = overlap[l]
        else:
          href_url = press[key][3]+overlap[l]
      
      
      href_resp = requests.get(href_url, headers=headers)
      href_soup = BeautifulSoup(href_resp.content, 'lxml')

      tit_tags = href_soup.select(press[key][4]) # 본문의 제목 태그
      day_tags = href_soup.select(press[key][5]) # 본문의 날짜 태그
      bon_tags = href_soup.select(press[key][6]) # 본문의 본문 태그

      
                  
      for tit, day in zip(tit_tags, day_tags):
        total_press.append(key)
        total_tit.append(tit.text)
        total_day.append(re.sub("[^0-9]", "", day.text)[:8])
        
        p_gather =[]      # 리스트 안에 리스트로 만들면서 0번째로 인식시키기 위함
        for bon in bon_tags:   # p태가 여러개이므로 제목 한개에 본문 한개로 인식시키기 위함
          if bon.text != '':
            p_gather.append(bon.text)

        total_bon.append(p_gather)

## 실행 코드 - > DataFrame -> DB화

In [ ]:
# 검색할 한글 키워드 리스트
key_list = ['캐논코리아', '캐논 카메라','소니 카메라','풀프레임', '미러리스', '카메라', '디지털카메라', '소니코리아', 'DSLR', '삼성 카메라'] # 캐논 소니 라이카 

# 아스키코드 저장할 리스트
ascii_list = []

# key_list를 아스키코드로 변환후 arkey_list에 append하는 함수
ascii_def = ascii(key_list)



# 언론사 딕셔너리
#   '언론사 이름' : 0. 검색했을때 나오는 url, 1. 검색했을때 나오는 제목의 a태그 , 2. 검색했을때 나오는 날짜 태그 , 
#                   3. 언론사 기본 url , 4. 본문의 제목 , 5. 본문의 날짜, 6.본문의 본문
press = {
    'aving':['https://kr.aving.net/news/articleList.html?page={0}&total=839&box_idxno=&sc_area=A&view_type=sm&sc_word={1}', 'div.view-cont h4.titles a', 'div.view-cont span.byline', 
             'http://kr.aving.net', 'h3.heading', 'div.view-info ul li', 'div.article-body p'],
    'zdnet':['https://search.zdnet.co.kr/?kwd={1}&area=&pageno={0}&term=3', 'div.assetText a', 'p.byline span', 
             'https://zdnet.co.kr', 'div.news_head h1', 'p.meta span', 'div.view_cont p'],
    '전자신문' : ['https://search.etnews.com/etnews/search.html?category=CATEGORY1&kwd={1}&pageNum={0}&pageSize=10&reSrchFlag=false&sort=1&startDate=\
        &endDate=&sitegubun=&jisikgubun=&preKwd%5B0%5D={1}', 'dl.clearfix dt a', 'div.list_search dd.date span',
                  'https://www.etnews.com', 'div.article_title h2', 'div.article_header_sub time.date', 'div.article_txt p'],
    '뉴시스' : ['https://newsis.com/search/?val={1}&sort=acc&jo=all_jogun&bun=all_bun&sdate=&term=allday&edate=&s_yn=Y&catg=1&t=1&page={0}&', 'ul.articleList2 p.tit a', 'div.txtCont p.time',
                'https://newsis.com', 'div.top p.tit.title_area', 'div.left p.txt', 'div.viewer article'],
    'IT동아' : ['https://it.donga.com/search/?page={0}&query={1}', 'li.media.my-4 a', 'div.mb-2.row div.col-sm-8.text-right time',
                  'https://it.donga.com/', 'header.mb-3.pb-0 h1', 'div.row.py-2 time', 'article.mb-3 p'],
    'IT조선' : ['https://it.chosun.com/svc/list_in/search.html?query={1}&sort=regdate&pn={0}', 'div.thumb_news div.txt_wrap > a', 'span.date',
              'http://it.chosun.com', 'div.news_title_text h1', 'div.news_date', 'div#news_body_id div.par'],
    '디지털타임스':['http://search2.dt.co.kr/newsList.jsp?section=news&txt={1}&f={0}&psection=1&sort=DATE/DESC','ul.s_tit a', 'div.dates',
                     'http://www.dt.co.kr','h1.top_title', 'li.lasttime', 'div.art_txt'],
    '헤럴드경제':['http://biz.heraldcorp.com/search/index.php?q={1}&sort=1&np={0}', 'div.list a', 'div.l_date',
                      'http://biz.heraldcorp.com', 'li.article_title.ellipsis2', 'li.article_date', 'div.article_view p'],
    '디지털데일리':['https://www.ddaily.co.kr/news/search_result.html?search={1}&search_mode=&hash=&start_date=&end_date=&page={0}', 'a.gray143c', 'span.font11blue2',
                    'https://www.ddaily.co.kr/news/', 'div.hbox h2', 'p.arvdate', 'div.smartOutput'],
    '스마트PC사랑' : ['https://www.ilovepc.co.kr/news/articleList.html?page={0}&total=251&box_idxno=&sc_area=A&view_type=sm&sc_word={1}', 'div.list-titles a', 'div.list-dated',
                      'https://www.ilovepc.co.kr', 'div.article-head-title', 'ul.no-bullet.auto-marbtm-0.line-height-6', 'div#article-view-content-div p']
    
    
    
    
}

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}


total_press, total_tit, total_day, total_bon = [], [], [], []

trend = news_trend(press)


dict_news = {
    '언론사':total_press,
    '제목':total_tit,
    '날짜':total_day,
    '본문':total_bon
} 

df_news = pd.DataFrame(dict_news)
df_news.to_csv("total_news.csv")


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/876 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2929 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/549 [00:00<?, ?it/s]

# 텍스트 마이닝

## 텍스트 전처리

### OKT preprocessing (전처리 함수)

In [ ]:
def preprocessing(trend):
    okt = Okt()
      
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    trend_text = re.sub("[^가-힣a-zA-Z\\s]", "", trend)
    
    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_trend = okt.pos(trend_text, stem=True) # stem = 어감추출 - 통일화?

    # 노이즈 & 불용어 제거
    word_trend = [(token,pos) for token, pos in word_trend if not token in stopwords_str and len(token) > 1 and len(token) <= 10] # if not token과 if token not 의 차이는 문법의 차이이다.

    # 명사, 동사, 형용사 추출
    word_trend = [token for token, pos in word_trend if pos in ['Noun','Alpha','Josa','Suffix','Adjective']]#, 'Verb', 'Adjective'

    return word_trend

### 불용어 사전 불러오기

In [ ]:
!gdown --id 1LVzS6N_YgtgelLGQinrW05nUNkJCfcYB

In [ ]:
stopwords = []

with open("트랜드_불용어사전2.txt", 'rt') as f:
  stopwords = f.read().split()
stopwords = list( set(stopwords) )
# stopwords

### 본문 csv파일을 2022년, 2021년 나누기

In [ ]:
# df_total_news = pd.read_csv('total_news.csv')

# df_total_news.astype({'날짜':'int64'}) # 날짜 컬럼의 데이터를 str에서 int로 변환
# df_total_news_2022 = df_total_news[df_total_news['날짜'] > 20220000 ] # 2022년 이상인 데이터를 저장
# df_total_news_2022 = df_total_news_2022.sort_values(by=['날짜']) # 날짜순으로 정령
# df_total_news_2022 = df_total_news_2022.reset_index(drop=True) # 필요없는 인덱스 컬럼 제거
# df_total_news_2022 = df_total_news_2022.drop(['Unnamed: 0'], axis=1) # 필요없는 인덱스 컬럼 제거

# df_total_news_2021 = df_total_news[df_total_news['날짜'] < 20220000 ]
# df_total_news_2021 = df_total_news_2021.sort_values(by=['날짜'])
# df_total_news_2021 = df_total_news_2021.reset_index(drop=True)
# df_total_news_2021 = df_total_news_2021.drop(['Unnamed: 0'], axis=1)
# #df_total_news_2021_2 = df_total_news[202106000 < df_total_news['날짜'] < 20220000 ]

# df_total_news_2022.to_csv("total_news_2022.csv")
# df_total_news_2021.to_csv("total_news_2021.csv")
# #df_total_news_2021_2.to_csv("total_news_2021_2.csv")

위의 크롤링 작업이 오래걸리므로 만들어진 파일 다운로드

In [ ]:
# !gdown --id 1TPezPmmZZLkTysv3GdIdrpfjwmhFLEFi

In [ ]:
# !unzip -qq "./total news.zip"

#### csv 파일이 5개일 경우 1개로 합치는 코드

빠른 작업 수행을 위한 적은 데이터 다운로드

In [ ]:
!gdown --id 1FznutQ2tIbm_CiR42N89XBH_R08eC8Rl

In [ ]:
!unzip -qq "./카메라 자료.zip"

In [ ]:
rs1 = pd.read_csv('total_news1.csv')
rs2 = pd.read_csv('total_news2.csv')
rs3 = pd.read_csv('total_news3.csv')
rs4 = pd.read_csv('total_news4.csv')
rs5 = pd.read_csv('total_news5.csv')

df_total_news_2022 = pd.concat([rs1,rs2,rs3,rs4,rs5], ignore_index=True)

### 키워드 분석

In [ ]:
from collections import Counter

stopwords_str = '\n'.join(stopwords)

# 22년 전처리
pre_total_2022 = []
df_total_news_2022['article_preprocessed'] = ''
for y in tqdm(range(len(df_total_news_2022['본문']))):
    word_trend = preprocessing(df_total_news_2022['본문'][y])
    pre_total_2022 += word_trend
    df_total_news_2022['article_preprocessed'].iloc[y] = ' '.join(word_trend)

# 21년 전처리
# pre_total_2021 = []
# df_total_news_2021['article_preprocessed'] = ''
# for z in tqdm(range(len(df_total_news_2021['본문']))):
#     word_trend = preprocessing(df_total_news_2021['본문'][z])
#     pre_total_2021 += word_trend
#     df_total_news_2021['article_preprocessed'].iloc[z] = ' '.join(word_trend)

# 22년 counter
word_con_2022 = Counter(pre_total_2022)
# 21년 counter
# word_con_2021 = Counter(pre_total_2021)

from tqdm import tqdm
tqdm.pandas() 

all_of_count_2022 = sorted(word_con_2022.items(), key=lambda x: x[1], reverse=True)
# all_of_count_2021 = sorted(word_con_2021.items(), key=lambda x: x[1], reverse=True)

word_ranking_2022 = dict(all_of_count_2022[:20])
# word_ranking_2021 = dict(all_of_count_2021[:20])
word_ranking_2022

  0%|          | 0/1467 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 1467/1467 [02:22<00:00, 10.30it/s]


## 키워드 시각화

### 라이브러리

In [ ]:
# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

!pip install chart_studio
# pandas에서 바로 plotly로 그래프를 제작해주는 라이브러리 
!pip install cufflinks

#pandas profiling
!pip install -U pandas-profiling

Selecting previously unselected package fonts-nanum.
(Reading database ... 155673 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Looking in indexes: https://pypi.org/simple, https://us-python

In [ ]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
import matplotlib as mpl

path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
# 폰트 지정
plt.rc('font', family=font_name)

#마이너스 깨짐현상
mpl.rc('axes', unicode_minus=False)

fm._rebuild()

NanumGothic Eco


### 히스토그램 그리기

In [ ]:
list_of_maker = ['word_ranking_2022']

list_of_df = []
list_of_dic = []

noun_count_list = [word_ranking_2022]

for i, maker in enumerate( list_of_maker ):
  tmp_dic = {
    'Word' : [],
    'Count' : []
  }

  for j, k in noun_count_list[i].items():

    tmp_dic['Word'].append( j )
    tmp_dic['Count'].append( k )

  tmp_df = pd.DataFrame( tmp_dic )
  list_of_dic.append(tmp_dic)
  list_of_df.append(tmp_df)
  tmp_df.to_csv( 'df_keyword_n_count_of_{0}' .format( maker ) )

for i, j in enumerate(list_of_df): #데이터프레임 출력
  print(list_of_maker[i])
  display(j)
  print()

word_ranking_2022


,Word,Count
0,기술,2167
1,제품,1803
2,촬영,1584
3,지원,1287
4,시장,1271
5,사진,1267
6,탑재,1254
7,영상,1229
8,기능,1199
9,렌즈,1146


In [ ]:
import plotly.express as px

for i in list_of_df:
  a = px.histogram( i, x='Word', y='Count', color='Word' )
  a.show()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## N_gram을 적용한 토픽모델링

### TF-IDF 행렬 만들기

In [ ]:
def my_tokenizer(text):
  return text.split()

In [ ]:
# sklearn을 통해서 TF-IDF 행렬을 만들어 줍니다. 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(2, 2))
tfidf = tfidf_vect.fit_transform(raw_documents = df_total_news_2022['article_preprocessed'].iloc[:]) # 기사 개수는 1000개로 조절하겠습니다

tfidf

<1467x210136 sparse matrix of type '<class 'numpy.float64'>'
	with 290072 stored elements in Compressed Sparse Row format>

### LDA 학습

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components =5, 
                                max_iter = 20,
                                learning_method='online',
                                random_state=100)
lda_output = lda.fit_transform(tfidf)

In [ ]:
print('lda components:\n',lda.components_) 
print('shape:', lda.components_.shape) 

lda components:
 [[0.20001384 0.20001829 0.20002269 ... 0.20002403 0.20002342 0.2000258 ]
 [0.20001697 0.20001972 0.20001903 ... 0.27202954 0.20002673 0.20002284]
 [0.20001499 0.20052942 0.20002372 ... 0.2000227  0.20002386 0.20002258]
 [0.20002809 0.26276044 0.20001879 ... 0.2000238  0.27667975 0.20002388]
 [0.28571959 0.20001601 0.27409211 ... 0.20002182 0.20002643 0.29857544]]
shape: (5, 210136)


### LDA 과정에서 얻은 토픽들 프린트, 토픽당 7개의 단어를 default로 설정

In [ ]:
terms = tfidf_vect.get_feature_names() # 단어 집합

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda.components_,terms)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Topic 1: [('is stm', 2.21), ('초점 렌즈', 1.69), ('cnemm ffp', 1.58), ('엔비디아 브로드캐스트', 1.48), ('rfmm is', 1.33), ('광각 렌즈', 1.26), ('배경 효과', 1.23)]
Topic 2: [('웰딩 ai', 1.61), ('버추 프로덕션', 1.51), ('인더스트리얼 ai', 1.49), ('불량 판별', 1.28), ('버설 ai', 1.27), ('인공 지능', 1.25), ('ai 코어', 1.23)]
Topic 3: [('용량 배터리', 1.9), ('유튜브 프리미엄', 1.81), ('화이트 가지', 1.79), ('mah 용량', 1.75), ('ai 반도체', 1.62), ('무료 체험', 1.62), ('구매 고객', 1.61)]
Topic 4: [('캐논 코리아', 5.45), ('매출 이익', 5.2), ('분기 매출', 4.02), ('eos eos', 3.15), ('미러리스 eos', 3.03), ('분기 실적', 2.99), ('eos 시스템', 2.88)]
Topic 5: [('이미지 센서', 8.25), ('캐논 코리아', 5.25), ('인공 지능', 3.18), ('동영상 촬영', 3.13), ('gb gb', 3.03), ('화소 이미지', 2.94), ('사진 촬영', 2.8)]
